In [1]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import glob
import os
from os import *
import re
import imutils
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import collections

In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]


In [3]:
p = '/home/jovyan/DATA_MASTER_PROJECT/A549/Differente_fields_of_view/test_full/'

t = []
for i in listdir(p):
    im =cv2.imread(p+i)
    t.append(im)

In [4]:
len(t)

60

In [5]:
def get_time_point(path, time_point_1,time_point_2 ):   # time_point in form of 'Take #1 ' (remember the space between #1 ') or 'Take #32'

    image_time_point = []

    for dirname, dirnames,filenames in os.walk(path):



        for filename in sorted(filenames, key = natural_keys):



            if time_point_1 in filename or time_point_2 in filename:



                im = cv2.imread(os.path.join(dirname, filename))



                image_time_point.append(im)

    return image_time_point   

In [6]:
# def get_time_point(path):   # time_point in form of 'Take #1 ' (remember the space between #1 ') or 'Take #32'

#     image_time_point = []

#     for i in listdir(path):
#         im =cv2.imread(path+i)
    
#         image_time_point.append(im)

#     return image_time_point   

In [7]:
def find_and_cut(path_img,time_point_1,time_point_2, save_path): #'Take #1 ', # "{0}/sk_{1}_exp_2_time_1.tiff"

    t_1 = get_time_point(path_img,time_point_1,time_point_2)

    tot_dict = {}

    for idx, image in enumerate(t_1):

        dict_1 = {}

        cx = []
        cy = []

        shifted = cv2.pyrMeanShiftFiltering(image, 10, 50)
        gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        D = ndimage.distance_transform_edt(thresh)
        localMax = peak_local_max(D, indices=False, min_distance=20, labels=thresh)
        markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
        labels = watershed(-D, markers, mask=thresh)



        for label in np.unique(labels):
            if label == 0:
                continue




            mask = np.zeros(gray.shape, dtype='uint8')
            mask[labels == label] = 255
            cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            cnts = imutils.grab_contours(cnts)
            c = max(cnts, key=cv2.contourArea)
            ((x, y), r) = cv2.minEnclosingCircle(c)



            cx.append(int(x))
            cy.append(int(y))


            data = {'X':cx,'Y':cy}

            data = pd.DataFrame(data)


        dict_1[idx] = data

        tot_dict.update(dict_1)


    t_1_1 = get_time_point(path_img, time_point_1,time_point_2)


    value_dict = []

    for k, v in sorted(tot_dict.items()):
        value_dict.append(v)



    crop_cell_1 = []

    for idx , i in enumerate(t_1_1):

        img = t_1_1[idx]



        x = []
        y = []

        for i in range(len(value_dict[idx])):
            x.append(value_dict[idx].iloc[i,0])      
            y.append(value_dict[idx].iloc[i,1])


        w = 45
        h = 45

        for ix, i in enumerate(x):

            crop = img[(y[ix]-h):(y[ix]+h), (x[ix]-w):(x[ix]+w)]

            if crop.size == 24300: 

                crop_cell_1.append(crop)





    for ls, i in enumerate(crop_cell_1):

        im = Image.fromarray(i)

        im.save(save_path.format(via, ls))   # exp 1 or 2 sonce they did dublicate studies 

    return crop_cell_1

In [8]:
via = '/home/jovyan/DATA_MASTER_PROJECT/A549/Differente_fields_of_view/cropped_test'
directory = os.mkdir(via) 

In [9]:
path_img = p


In [12]:
croped_cell = find_and_cut(path_img, 'E3','G3' , "{0}/controll{1}_.tiff")

len(croped_cell)

732

In [13]:


a0 = [['controll', 0]] * 732

a1 =[['dilamrate', 1]] * 2206

a2 =[['peroxide', 2]] * 918

#a3 =[['mast', 3]] * 50

# a4 =[['misoprostol', 4]] * 8371

# a5 =[['molindone', 5]] * 6766

# a6 =[['nadol', 6]] * 7420

# a7 =[['nadol', 7]] * 7420



labels = a1+a0+a2#+a2#+a2+a3+a4+a5+a6



In [14]:
df = pd.DataFrame(labels, columns = ['Type', 'Category'])
df = df.sort_values(by=['Type'])

In [15]:

df.to_csv("/home/jovyan/DATA_MASTER_PROJECT/A549/Differente_fields_of_view/test_labels.csv", sep=',',index=False)